In [1]:
import pandas as pd
import numpy as np
#import pandas_profiling
import seaborn as sns
from scipy import stats
from matplotlib import pyplot as plt
from sklearn import preprocessing
from sklearn.cluster import KMeans
%matplotlib inline
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
from pandas import read_csv, DataFrame
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from catboost import CatBoostRegressor, cv
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from IPython.display import display
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error, max_error

In [2]:
# Задаем рабочую папку
import os
os.chdir(r"C:\Users\Ckomap\Desktop\Вкр")

In [3]:
#Загрузим ранее полученные датафреймы
df = pd.read_csv('df.csv')
df_norm = pd.read_csv('df_norm.csv')
df_0 = pd.read_csv('df_0.csv')
df_1 = pd.read_csv('df_1.csv')

In [15]:
##Разобьем данные на тестовую и итоговую выборки 
#df_norm
x_norm = df_norm[['Соотношение матрица-наполнитель','Угол нашивки, град','модуль упругости, ГПа','Плотность, кг/м3','Количество отвердителя, м.%','Содержание эпоксидных групп,%_2','Температура вспышки, С_2','Поверхностная плотность, г/м2','Потребление смолы, г/м2','Шаг нашивки','Плотность нашивки']]
y_norm = df_norm[['Прочность при растяжении, МПа','Модуль упругости при растяжении, ГПа']]
x_norm_train, x_norm_test, y_norm_train, y_norm_test = train_test_split(x_norm,y_norm,test_size=0.3,random_state=42)
#df_0
x_0 = df_0[['Соотношение матрица-наполнитель','Угол нашивки, град','модуль упругости, ГПа','Плотность, кг/м3','Количество отвердителя, м.%','Содержание эпоксидных групп,%_2','Температура вспышки, С_2','Поверхностная плотность, г/м2','Потребление смолы, г/м2','Шаг нашивки','Плотность нашивки']]
y_0 = df_0[['Прочность при растяжении, МПа','Модуль упругости при растяжении, ГПа']]
x_0_train, x_0_test, y_0_train, y_0_test = train_test_split(x_0,y_0,test_size=0.3,random_state=42)
#df_1
x_1 = df_1[['Соотношение матрица-наполнитель','Угол нашивки, град','модуль упругости, ГПа','Плотность, кг/м3','Количество отвердителя, м.%','Содержание эпоксидных групп,%_2','Температура вспышки, С_2','Поверхностная плотность, г/м2','Потребление смолы, г/м2','Шаг нашивки','Плотность нашивки']]
y_1 = df_1[['Прочность при растяжении, МПа','Модуль упругости при растяжении, ГПа']]
x_1_train, x_1_test, y_1_train, y_1_test = train_test_split(x_1,y_1,test_size=0.3,random_state=42)
#Сделаем предположение об использовании только 6 наиболее значимых параметров
#df_norm
x_6 = df_norm[['Плотность, кг/м3','Количество отвердителя, м.%', 'Содержание эпоксидных групп,%_2',
               'Температура вспышки, С_2','Потребление смолы, г/м2','Плотность нашивки']]
y_6 = df_norm[['Прочность при растяжении, МПа','Модуль упругости при растяжении, ГПа']]
x_6_train, x_6_test, y_6_train, y_6_test = train_test_split(x_6,y_6,test_size=0.3,random_state=42)
#df_0
x_06 = df_0[['Плотность, кг/м3','Количество отвердителя, м.%', 'Содержание эпоксидных групп,%_2',
               'Температура вспышки, С_2','Потребление смолы, г/м2','Плотность нашивки']]
y_06 = df_0[['Прочность при растяжении, МПа','Модуль упругости при растяжении, ГПа']]
x_06_train, x_06_test, y_06_train, y_06_test = train_test_split(x_06,y_06,test_size=0.3,random_state=42)
#df_1
x_16 = df_1[['Плотность, кг/м3','Количество отвердителя, м.%', 'Содержание эпоксидных групп,%_2',
               'Температура вспышки, С_2','Потребление смолы, г/м2','Плотность нашивки']]
y_16 = df_1[['Прочность при растяжении, МПа','Модуль упругости при растяжении, ГПа']]
x_16_train, x_16_test, y_16_train, y_16_test = train_test_split(x_16,y_16,test_size=0.3,random_state=42)
#Создадим словарь с наборами данных
xy_list = {'Полный датасет' : [x_norm_train, x_norm_test, y_norm_train, y_norm_test], 
           '0-й кластер' : [x_0_train, x_0_test, y_0_train, y_0_test],
           '1-й кластер' : [x_1_train, x_1_test, y_1_train, y_1_test],
           'Полный датасет 6 столбцов' : [x_6_train, x_6_test, y_6_train, y_6_test],
           '0-й кластер 6 столбцов' : [x_06_train, x_06_test, y_06_train, y_06_test], 
           '1-й клстер 6 столбцов' : [x_16_train, x_16_test, y_16_train, y_16_test]
          }
metrics_tmp = ['train_r2', 'test_r2', 'mse', 'mae', 'mape']

In [5]:
# Создадим датафрейм для занесения в него метрик разных датафремов методом Лассо
results = pd.DataFrame(np.zeros((5, 6)), columns = xy_list.keys(), index = metrics_tmp)
results

,Полный датасет,0-й кластер,1-й кластер,Полный датасет 6 столбцов,0-й кластер 6 столбцов,1-й клстер 6 столбцов
train_r2,0.0,0.0,0.0,0.0,0.0,0.0
test_r2,0.0,0.0,0.0,0.0,0.0,0.0
mse,0.0,0.0,0.0,0.0,0.0,0.0
mae,0.0,0.0,0.0,0.0,0.0,0.0
mape,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
#Рассмотрим применение Случайного Леса для разных наборов данных
def model_rf(x_train, x_test, y_train, y_test):
    model = RandomForestRegressor()
    model.fit(x_train, y_train) #обучим модель
    predict = model.predict(x_train) 
    y_predict = model.predict(x_test) #предсказание модели
    train_r2 = metrics.r2_score(y_train, predict)# коэффициент детерминации r2 train
    test_r2 = metrics.r2_score(y_test, y_predict)  # коэффициент детерминации r2 test
    mse = mean_squared_error(y_test, y_predict)   # среднеквадратичная ошибка
    mae = mean_absolute_error(y_test, y_predict) #средняя абсолютная ошибка
    mape = mean_absolute_percentage_error(y_test, y_predict)  #Средняя абсолютная ошибка в процентах
    return [train_r2, test_r2, mse, mae, mape]

In [7]:
#Заполним таблицу испытаний Случайного Леса
rf_results = results.copy()
for i in xy_list:
    rf_results[i] =model_rf(*xy_list[i])
rf_results.T

,train_r2,test_r2,mse,mae,mape
Полный датасет,0.854960,-0.032283,0.029127,0.136062,3.782860e+12
0-й кластер,0.850022,-0.008196,0.029207,0.133658,7.030485e+12
1-й кластер,0.859613,0.019939,0.024671,0.126981,3.546606e-01
Полный датасет 6 столбцов,0.853679,-0.044303,0.029447,0.136718,3.523526e+12
0-й кластер 6 столбцов,0.852872,-0.071549,0.031044,0.138056,7.496660e+12
1-й клстер 6 столбцов,0.852201,0.030869,0.024396,0.126096,3.509530e-01


In [8]:
def lasso_reg(x_train, x_test, y_train, y_test):
    # строим модель, обучаем ее, проверяем прогнозные значения, смотрим метрики
    lasso = Lasso()
    lasso.fit(x_train, y_train)
    predict = lasso.predict(x_train)
    # коэффициент детерминации, т.е. смотрим насколько хорошо обучена модель
    train_r2 = metrics.r2_score(y_train, predict) 
    y_predict = lasso.predict(x_test)
    test_r2 = metrics.r2_score(y_test, y_predict) 
    mse = mean_squared_error(y_test, y_predict)
    mae = mean_absolute_error(y_test, y_predict)
    mape = mean_absolute_percentage_error(y_test, y_predict)
    return [train_r2, test_r2, mse, mae, mape]

In [9]:
#Заполним таблицу испытаний Лассо
lasso_results = results.copy()
for i in xy_list:
    lasso_results[i] =lasso_reg(*xy_list[i])
lasso_results.T

,train_r2,test_r2,mse,mae,mape
Полный датасет,2.220446e-16,-0.009483,0.028476,0.132860,3.810883e+12
0-й кластер,-5.551115e-16,-0.003623,0.029093,0.134513,8.189343e+12
1-й кластер,3.885781e-16,-0.003551,0.025251,0.125529,3.714942e-01
Полный датасет 6 столбцов,2.220446e-16,-0.009483,0.028476,0.132860,3.810883e+12
0-й кластер 6 столбцов,-5.551115e-16,-0.003623,0.029093,0.134513,8.189343e+12
1-й клстер 6 столбцов,3.885781e-16,-0.003551,0.025251,0.125529,3.714942e-01


In [10]:
###Градиентный Бустинг
# по аналогии с предыдущими функциями оформим Градиентный Бустинг
def gbr_model(x_train, x_test, y_train, y_test):
    model = GradientBoostingRegressor()
    model.fit(x_train,y_train) 
    predict = model.predict(x_train) 
    train_r2 = metrics.r2_score(y_train, predict) 
    y_predict = model.predict(x_test)
    test_r2 = metrics.r2_score(y_test, y_predict) 
    mse = mean_squared_error(y_test, y_predict)
    mae = mean_absolute_error(y_test, y_predict)
    mape = mean_absolute_percentage_error(y_test, y_predict)
    return [train_r2, test_r2, mse, mae, mape]

In [11]:
#Заполним таблицу испытаний Лассо
gbr_results = results.copy()
for i in xy_list:
    gbr_results[i] =lasso_reg(*xy_list[i])
gbr_results.T

,train_r2,test_r2,mse,mae,mape
Полный датасет,2.220446e-16,-0.009483,0.028476,0.132860,3.810883e+12
0-й кластер,-5.551115e-16,-0.003623,0.029093,0.134513,8.189343e+12
1-й кластер,3.885781e-16,-0.003551,0.025251,0.125529,3.714942e-01
Полный датасет 6 столбцов,2.220446e-16,-0.009483,0.028476,0.132860,3.810883e+12
0-й кластер 6 столбцов,-5.551115e-16,-0.003623,0.029093,0.134513,8.189343e+12
1-й клстер 6 столбцов,3.885781e-16,-0.003551,0.025251,0.125529,3.714942e-01
